In [1]:
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, clear_output
from PIL import Image as PILImage

# Initialize MediaPipe Pose and Drawing
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Define Utility Functions
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
    
    return angle

def get_coordinates(landmarks, body_part):
    return [landmarks.landmark[body_part].x, landmarks.landmark[body_part].y]

# Define Exercise Analysis Functions
def analyze_raise_hand(landmarks):
    global raise_hand_counter, prev_feedback
    shoulder = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER.value)
    elbow = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_ELBOW.value)
    wrist = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_WRIST.value)
    
    angle_elbow_wrist = calculate_angle(shoulder, elbow, wrist)
    
    if 120 <= angle_elbow_wrist <= 180:
        feedback = "Good job"
    else:
        feedback = "Raise your hand higher"
    
    # Update counter only when feedback changes
    if feedback != prev_feedback:
        raise_hand_counter += 1
        prev_feedback = feedback
    
    return feedback

def analyze_chest_exercise(landmarks):
    global chest_exercise_counter, prev_feedback
    shoulder = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER.value)
    elbow = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_ELBOW.value)
    wrist = get_coordinates(landmarks, mp_pose.PoseLandmark.LEFT_WRIST.value)
    
    angle_shoulder_elbow = calculate_angle(shoulder, elbow, wrist)
    
    if 160 < angle_shoulder_elbow < 180:
        feedback = "Good chest exercise"
    else:
        feedback = "Maintain proper form"
    
    # Update counter only when feedback changes
    if feedback != prev_feedback:
        chest_exercise_counter += 1
        prev_feedback = feedback
    
    return feedback

# Map exercise names to analysis functions
exercise_functions = {
    "Raise Hand": analyze_raise_hand,
    "Chest Exercise": analyze_chest_exercise,
}

# Select Exercise Type
exercise_type = input("Select exercise (Raise Hand, Chest Exercise): ")

# Initialize Video Capture
cap = cv2.VideoCapture(0)

# Initialize counters
raise_hand_counter = 0
chest_exercise_counter = 0
prev_feedback = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks
        feedback = exercise_functions[exercise_type](landmarks)
        mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Display feedback on the frame
        cv2.putText(frame, feedback, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display counter value
        cv2.putText(frame, f"Counter: {raise_hand_counter if exercise_type == 'Raise Hand' else chest_exercise_counter}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the frame in Jupyter Notebook
    pil_img = PILImage.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    display(pil_img)
    clear_output(wait=True)

cap.release()

Select exercise (Raise Hand, Chest Exercise):  Chest Exercise
